### Question 4 

### (i)

In [ ]:
import numpy as np

# Given option information
S0 = 58  # Spot price
K = 60   # Strike price
T = 9/12  # Time to maturity (9 months)
r = 0.02  # Risk-free interest rate
q = 0.01  # Dividend rate
option_price = 6.36  # Given option price
tolerance = 1e-4  # Tolerance level for convergence
sigma = 0.353835  # randomly chosen volatility

# Identify the number of time steps Nfixed in the binomial tree pricer. Start with 10 time
# steps for the given option and double the time steps until two consecutive approximations are
# within tolerance of each other. The finest tree considered corresponds to Nfixed time steps.

N = 10  # Initial number of time steps
prev = 0  # Previous option price
while True:
    dt = T / N  # Time step
    u = np.exp(sigma*np.sqrt(dt))  # Up factor
    d = 1/u
    p = (np.exp((r - q) * dt) - d) / (u - d)  # Risk-neutral probability

    # Initialize the stock price tree
    S = np.zeros((N + 1, N + 1))
    S[0, 0] = S0
    for i in range(1, N + 1):
        S[i, 0] = S[i - 1, 0] * u
        for j in range(1, i + 1):
            S[i, j] = S[i - 1, j - 1] * d

    # Initialize the option price tree
    V = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        V[N, j] = max(S[N, j] - K, 0)

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            V[i, j] = np.exp(-r * dt) * (p * V[i + 1, j] + (1 - p) * V[i + 1, j + 1])

    # Check if the option price converges
    print(V[0,0])
    if abs(V[0, 0] - prev) < tolerance:
        break
    else:
        N *= 2
        prev = V[0, 0]

# final iteration of the binomial tree pricer
N_fixed = N

print('The number of time steps in the finest tree is', N_fixed)
print('The option price is', V[0, 0])

### (ii)

In [46]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

S0 = 58  # Spot price
K = 60   # Strike price
T = 9/12  # Time to maturity (9 months)
r = 0.02  # Risk-free interest rate
q = 0.01  # Dividend rate
option_price = 6.36  # Given option price

tolerance = 1e-4  # Tolerance level for convergence

sigma0 = 0.05  # Initial volatility
sigma1 = 1  # Initial volatility
prev = 0  # Previous option price


res = pd.DataFrame(columns=['sigma', 'option_price'])

# we use the fixed number of nodes in the previous that makes binomial tree pricer converges
# N = N_fixed
N = 2560
while abs(sigma0-sigma1) > tolerance:
    dt = T / N  # Time step
    u = np.exp(sigma1*np.sqrt(dt))  # Up factor
    d = 1/u
    p = (np.exp((r - q) * dt) - d) / (u - d)  # Risk-neutral probability
    
    # Initialize the stock price tree
    S = np.zeros((N + 1, N + 1))
    S[0, 0] = S0
    for i in range(1, N + 1):
        S[i, 0] = S[i - 1, 0] * u
        for j in range(1, i + 1):
            S[i, j] = S[i - 1, j - 1] * d

    # Initialize the option price tree
    V = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        V[N, j] = max(S[N, j] - K, 0)

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            V[i, j] = np.exp(-r * dt) * (p * V[i + 1, j] + (1 - p) * V[i + 1, j + 1])

    print(sigma1, V[0,0])
    res = res.append({'sigma': sigma1, 'option_price': V[0,0]}, ignore_index=True)

    # secant method
    sigma = sigma1 - (sigma1-sigma0)*(V[0,0]-option_price)/(V[0,0]-prev)
    sigma0 = sigma1
    sigma1 = sigma

    prev = V[0, 0]
print('The implied volatility is', sigma)
res


1 18.779176474068397
0.3717393482799002 6.715533619445228
0.35322356609430083 6.347867778532798
0.3538345490715518 6.3600087448748885
The implied volatility is 0.353834108993757


,sigma,option_price
0,1.0,18.779176
1,0.371739,6.715534
2,0.353224,6.347868
3,0.353835,6.360009
